In [1]:
import pandas as pd
import locale
import numpy as np
locale.setlocale(locale.LC_ALL, '')


'en_US.UTF-8'

In [2]:
# reading in my data files
school_data_file = 'Resources/schools_complete.csv'
student_data_file = 'Resources/students_complete.csv'

schools = pd.read_csv(school_data_file)
students = pd.read_csv(student_data_file)


In [3]:
# just checking the tables
schools.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [4]:
# playing with some custom filters
students[(students['student_name'].str.contains('Smith'))&\
         (students['math_score']<70)|\
         (students['reading_score']>98)\
        ].head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
1,1,Victor Smith,M,12th,Huang High School,94,61
22,22,Kimberly Stewart,F,11th,Huang High School,99,84
35,35,Benjamin Carlson,M,10th,Huang High School,99,61
53,53,Jon Smith,M,9th,Huang High School,96,69
78,78,Jason Thompson,M,12th,Huang High School,99,57


In [5]:
# merging the tables 
students_schools = pd.merge(students,schools,
                            on='school_name',
                            how='outer',
                            left_on=None,
                            right_on=None,
                            left_index=False,
                            right_index=False,
                            sort=True,
                            suffixes=('_students', '_schools'),
                            copy=True,
                            indicator=False,
                            validate='m:m',
                           )
students_schools.head(3)


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,17871,Blake Martin,M,9th,Bailey High School,75,59,7,District,4976,3124928
1,17872,Kathryn Kane,F,12th,Bailey High School,84,58,7,District,4976,3124928
2,17873,Richard Haas,M,11th,Bailey High School,79,86,7,District,4976,3124928


### District Summary

* Create a high level snapshot (in table form) of the district's key metrics, including:
  * Total Schools
  * Total Students
  * Total Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)


In [6]:
# renaming columns
old_cols = students_schools.columns
new_cols = ['Student ID', 'Student', 'Gender', 'Grade Level', 'School', 'Reading', 'Math', 
            'School ID', 'School Type', 'School Size', 'School Budget']
new_columns = dict(zip(old_cols,new_cols))
students_schools.rename(columns=new_columns,inplace=True)

students_schools.head()

,Student ID,Student,Gender,Grade Level,School,Reading,Math,School ID,School Type,School Size,School Budget
0,17871,Blake Martin,M,9th,Bailey High School,75,59,7,District,4976,3124928
1,17872,Kathryn Kane,F,12th,Bailey High School,84,58,7,District,4976,3124928
2,17873,Richard Haas,M,11th,Bailey High School,79,86,7,District,4976,3124928
3,17874,Frank Marsh,M,9th,Bailey High School,71,89,7,District,4976,3124928
4,17875,Charles Goodman Jr.,M,9th,Bailey High School,90,61,7,District,4976,3124928


In [7]:
# establishing some variables for targeted data

total_schools = len(students_schools['School'].unique())
total_students = len(students_schools['Student ID'].unique())

get_budget = pd.DataFrame(students_schools.groupby('School').max())
total_budget = get_budget['School Budget'].sum()

avg_math = students_schools['Math'].mean()
avg_read = students_schools['Reading'].mean()

pass_math = students_schools.loc[students_schools['Math'] >= 60]['Student'].count()
pass_read = students_schools.loc[students_schools['Reading'] >= 60]['Student'].count()

math_rate = round(pass_math/total_students*100,2)
read_rate = round(pass_read/total_students*100,2)
avg_pass = round((math_rate+read_rate)/2,2)



In [8]:
# another way to find passing rates
students_schools['Pass Math'] = np.where(students_schools['Math']>=60,1,0)
students_schools['Pass Reading'] = np.where(students_schools['Reading']>=60,1,0)
students_schools['Pass Both'] = np.where((students_schools['Pass Math']+students_schools['Pass Reading'])==2,1,0)



In [9]:
# creating some bins to add Grades to math and reading scores.
bins = [0, 59, 69, 79, 89, 100]
grades = ["F", "D", "C", "B", "A"]
students_schools['Math Grade'] = pd.cut(students_schools['Math'], bins, labels=grades)
students_schools['Reading Grade'] = pd.cut(students_schools['Reading'], bins, labels=grades)
failing_students = students_schools.loc[(students_schools['Math Grade']=='F') & (students_schools['Reading Grade']=='D')]


In [10]:
# putting it all together in a district summary table.
district_summary = pd.DataFrame({"Total Schools":[total_schools],
                                "Total Students":locale.format_string("%.f",total_students,grouping=True),
                              "Total Budget":locale.currency(total_budget,grouping=True),
                               "Average Math Score":round(avg_math,2),
                               "Average Reading Score":round(avg_read,2),
                               "Math Pass Rate":math_rate,
                               "Reading Pass Rate":read_rate,
                               "Total Pass Rate":avg_pass
                               
                              })

# outputing to a file
district_summary.to_csv("Output/district_summary.csv", index=False, header=True)
district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Math Pass Rate,Reading Pass Rate,Total Pass Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,92.45,100.0,96.22


### School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)


In [11]:
# using the groupby function to create new data columns in a summarized table.  
# this feels like the long way...

# count total students per School
school_summary = students_schools.groupby(['School Type','School'],as_index=False)\
                        ['Student'].count()\
                        .rename(columns={'Student':'Total Students'})

# show the budget for each School.
# since every instance is repeated, I used max.  probably could use .unique() as well
school_summary1 = pd.merge(school_summary,students_schools.groupby(['School'],as_index=False)\
                           ['School Budget'].max()\
                           ,on='School')

# show the average Math score per School
school_summary2 = pd.merge(school_summary1,students_schools.groupby(['School'],as_index=False)\
                           ['Math'].mean()\
                           .rename(columns={'Math':'Average Math Score'})\
                           ,on='School')
# show the average Reading score per School
school_summary3 = pd.merge(school_summary2,students_schools.groupby(['School'],as_index=False)\
                           ['Reading'].mean()\
                           .rename(columns={'Reading':'Average Reading Score'})\
                           ,on='School')

# create a new calculated field to find the budget per student
school_summary3['Budget per Student'] = (school_summary3['School Budget']//school_summary3['Total Students'])


In [12]:
# adding some Math stats using my 'Math flags'
school_summary4 = pd.merge(school_summary3,students_schools.groupby(['School'],as_index=False)\
                           ['Pass Math'].sum()\
                           .rename(columns={'Pass Math':'Passed Math'})\
                           ,on='School')

school_summary4['Passed Math Rate'] = round(school_summary4['Passed Math']/school_summary4['Total Students']*100,2)

school_summary4.head()

,School Type,School,Total Students,School Budget,Average Math Score,Average Reading Score,Budget per Student,Passed Math,Passed Math Rate
0,Charter,Cabrera High School,1858,1081356,83.061895,83.975780,582,1858,100.0
1,Charter,Griffin High School,1468,917500,83.351499,83.816757,625,1468,100.0
2,Charter,Holden High School,427,248087,83.803279,83.814988,581,427,100.0
3,Charter,Pena High School,962,585858,83.839917,84.044699,609,962,100.0
4,Charter,Shelton High School,1761,1056600,83.359455,83.725724,600,1761,100.0


In [13]:
# adding some Reading stats using my 'Reading flags'
school_summary5 = pd.merge(school_summary4,students_schools.groupby(['School'],as_index=False)\
                           ['Pass Reading'].sum()\
                           .rename(columns={'Pass Reading':'Passed Reading'})\
                           ,on='School')

school_summary5['Passed Reading Rate'] = round(school_summary5['Passed Reading']/school_summary5['Total Students']*100,2)

school_summary5.head(3)

,School Type,School,Total Students,School Budget,Average Math Score,Average Reading Score,Budget per Student,Passed Math,Passed Math Rate,Passed Reading,Passed Reading Rate
0,Charter,Cabrera High School,1858,1081356,83.061895,83.975780,582,1858,100.0,1858,100.0
1,Charter,Griffin High School,1468,917500,83.351499,83.816757,625,1468,100.0,1468,100.0
2,Charter,Holden High School,427,248087,83.803279,83.814988,581,427,100.0,427,100.0


In [14]:

# added some 'Passed Both' stats
school_summary6 = pd.merge(school_summary5,students_schools.groupby(['School'],as_index=False)\
                           ['Pass Both'].sum()\
                           .rename(columns={'Pass Both':'Passed Both'})\
                           ,on='School')

school_summary6['Passing Rate'] = round(school_summary6['Passed Both']/school_summary6['Total Students']*100,2)
school_summary6['Averge Passing Rate'] = round((school_summary6['Passed Math Rate']+school_summary6['Passed Reading Rate'])/2,2)

# rounding off some decimals
school_summary6['Average Math Score'] = school_summary6['Average Math Score'].map("{:.2f}".format).astype(float)
school_summary6['Average Reading Score'] = school_summary6['Average Reading Score'].map("{:.2f}".format).astype(float)
school_summary6['School Budget'] = school_summary6['School Budget'].map('{:.2f}'.format).astype(float)


# send it to a file

school_summary6


,School Type,School,Total Students,School Budget,Average Math Score,Average Reading Score,Budget per Student,Passed Math,Passed Math Rate,Passed Reading,Passed Reading Rate,Passed Both,Passing Rate,Averge Passing Rate
0,Charter,Cabrera High School,1858,1081356.0,83.06,83.98,582,1858,100.00,1858,100.0,1858,100.00,100.00
1,Charter,Griffin High School,1468,917500.0,83.35,83.82,625,1468,100.00,1468,100.0,1468,100.00,100.00
2,Charter,Holden High School,427,248087.0,83.80,83.81,581,427,100.00,427,100.0,427,100.00,100.00
3,Charter,Pena High School,962,585858.0,83.84,84.04,609,962,100.00,962,100.0,962,100.00,100.00
4,Charter,Shelton High School,1761,1056600.0,83.36,83.73,600,1761,100.00,1761,100.0,1761,100.00,100.00
5,Charter,Thomas High School,1635,1043130.0,83.42,83.85,638,1635,100.00,1635,100.0,1635,100.00,100.00
6,Charter,Wilson High School,2283,1319574.0,83.27,83.99,578,2283,100.00,2283,100.0,2283,100.00,100.00
7,Charter,Wright High School,1800,1049400.0,83.68,83.95,583,1800,100.00,1800,100.0,1800,100.00,100.00
8,District,Bailey High School,4976,3124928.0,77.05,81.03,628,4455,89.53,4976,100.0,4455,89.53,94.76
9,District,Figueroa High School,2949,1884411.0,76.71,81.16,639,2608,88.44,2949,100.0,2608,88.44,94.22


In [15]:
# establishing new summary stats from the summary table.
Total_Schools = school_summary6['School'].count()
Total_Students = school_summary6['Total Students'].sum()
Total_Budget = school_summary6['School Budget'].sum()
Avg_Math_Score = school_summary6['Average Math Score'].mean()
Avg_Reading_Score = school_summary6['Average Reading Score'].mean()

In [16]:
# experimentig with multiple groups applying multiple agg functions.
school_summary1 = students_schools\
.groupby(['School Type','School','Grade Level'],as_index=False)\
[['School Budget','Student']]\
.agg(['max','count','sum'])

## Re-doing the School Summary with Cleaner Code



In [33]:
# Re-doing the School Summary with the agg function
School_Summary = students_schools.groupby('School',as_index=False).agg({
    'Student':'count',
    'School Budget':'first',
    'Math':'mean',
    'Reading':'mean',
    'Pass Math':'sum',
    'Pass Reading':'sum',
})

# DUDE!  That was so much clean.  Thanks Glen!
School_Summary['Budget per Student'] = School_Summary['School Budget']//School_Summary['Student']
School_Summary['% Math Passed'] = School_Summary['Pass Math']/School_Summary['Student']*100
School_Summary['% Reading Passed'] = School_Summary['Pass Reading']/School_Summary['Student']*100

School_Summary.to_csv("Output/School Summary.csv", index=False, header=True)
School_Summary

,School,Student,School Budget,Math,Reading,Pass Math,Pass Reading,Budget per Student,% Math Passed,% Reading Passed
0,Bailey High School,4976,3124928,77.048432,81.033963,4455,4976,628,89.529743,100.0
1,Cabrera High School,1858,1081356,83.061895,83.975780,1858,1858,582,100.000000,100.0
2,Figueroa High School,2949,1884411,76.711767,81.158020,2608,2949,639,88.436758,100.0
3,Ford High School,2739,1763916,77.102592,80.746258,2446,2739,644,89.302665,100.0
4,Griffin High School,1468,917500,83.351499,83.816757,1468,1468,625,100.000000,100.0
5,Hernandez High School,4635,3022020,77.289752,80.934412,4129,4635,652,89.083064,100.0
6,Holden High School,427,248087,83.803279,83.814988,427,427,581,100.000000,100.0
7,Huang High School,2917,1910635,76.629414,81.182722,2592,2917,655,88.858416,100.0
8,Johnson High School,4761,3094650,77.072464,80.966394,4246,4761,650,89.182945,100.0
9,Pena High School,962,585858,83.839917,84.044699,962,962,609,100.000000,100.0


In [18]:
# experimenting with a student report
student_report = students_schools.groupby(['School Type','School','Grade Level','Gender','Pass Both','Math Grade','Reading Grade'])['Student'].count()
student_report.to_csv("Output/student_report.csv", index=True, header=True)
student_report.head(5)

School Type  School               Grade Level  Gender  Pass Both  Math Grade  Reading Grade
Charter      Cabrera High School  10th         F       1          D           C                 5
                                                                              B                 6
                                                                              A                 3
                                                                  C           D                 2
                                                                              C                20
Name: Student, dtype: int64

### Top Performing Schools (By Passing Rate)

* Create a table that highlights the top 5 performing schools based on Overall Passing Rate. Include:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)


In [19]:
# setting some column scenarios
perf_cols = ['School', 'School Type', 'Total Students', 'School Budget', 'Budget per Student',
       'Average Math Score', 'Average Reading Score', 'Passed Math Rate', 'Passed Reading Rate', 'Passing Rate']

sort_cols = ['Passing Rate','Average Reading Score','Average Math Score']



In [20]:
# sorting summary table to find top performing schools

top_performing = school_summary6[perf_cols].sort_values(by=sort_cols,ascending=False).reset_index(drop=True)
top_performing['School Budget'] = top_performing['School Budget'].map("${:,.0f}".format)
top_performing['Budget per Student'] = top_performing['Budget per Student'].map("${:,.0f}".format)
top_performing.head(5)


,School,School Type,Total Students,School Budget,Budget per Student,Average Math Score,Average Reading Score,Passed Math Rate,Passed Reading Rate,Passing Rate
0,Pena High School,Charter,962,"$585,858",$609,83.84,84.04,100.0,100.0,100.0
1,Wilson High School,Charter,2283,"$1,319,574",$578,83.27,83.99,100.0,100.0,100.0
2,Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,100.0,100.0,100.0
3,Wright High School,Charter,1800,"$1,049,400",$583,83.68,83.95,100.0,100.0,100.0
4,Thomas High School,Charter,1635,"$1,043,130",$638,83.42,83.85,100.0,100.0,100.0


### Bottom Performing Schools (By Passing Rate)

* Create a table that highlights the bottom 5 performing schools based on Overall Passing Rate. Include all of the same metrics as above.


In [21]:
# sorting summary table to find bottom performing schools
bottom_performing = top_performing.sort_values(by=sort_cols,ascending=True).reset_index(drop=True)
bottom_performing.head(5)

,School,School Type,Total Students,School Budget,Budget per Student,Average Math Score,Average Reading Score,Passed Math Rate,Passed Reading Rate,Passing Rate
0,Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,88.44,100.0,88.44
1,Rodriguez High School,District,3999,"$2,547,363",$637,76.84,80.74,88.55,100.0,88.55
2,Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,88.86,100.0,88.86
3,Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,89.08,100.0,89.08
4,Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,89.18,100.0,89.18


### Math Scores by Grade\*\*

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.


In [22]:
# going back to the original merged table prepare a table that can be pivoted by Grade Level

grade_scores = students_schools.groupby(['School','Grade Level'], as_index=False)['Math','Reading'].mean()\
.rename(columns={'Math':'Avg Math Score','Reading':'Avg Reading Score'})
grade_scores['Avg Math Score'] = grade_scores['Avg Math Score'].astype(float).map("{:.2f}".format)
grade_scores['Avg Reading Score'] = grade_scores['Avg Reading Score'].astype(float).map("{:.2f}".format)
grade_scores.head()

,School,Grade Level,Avg Math Score,Avg Reading Score
0,Bailey High School,10th,77.00,80.91
1,Bailey High School,11th,77.52,80.95
2,Bailey High School,12th,76.49,80.91
3,Bailey High School,9th,77.08,81.30
4,Cabrera High School,10th,83.15,84.25


In [23]:
# Average Math grades per School by Grade Level
math_scores = grade_scores.pivot(index='School',columns='Grade Level',values='Avg Math Score')
math_scores.head()

Grade Level,10th,11th,12th,9th
School,,,,
Bailey High School,77.00,77.52,76.49,77.08
Cabrera High School,83.15,82.77,83.28,83.09
Figueroa High School,76.54,76.88,77.15,76.40
Ford High School,77.67,76.92,76.18,77.36
Griffin High School,84.23,83.84,83.36,82.04


### Reading Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.


In [24]:
reading_scores = grade_scores.pivot(index='School',columns='Grade Level',values='Avg Reading Score')
reading_scores.head()

Grade Level,10th,11th,12th,9th
School,,,,
Bailey High School,80.91,80.95,80.91,81.30
Cabrera High School,84.25,83.79,84.29,83.68
Figueroa High School,81.41,80.64,81.38,81.20
Ford High School,81.26,80.40,80.66,80.63
Griffin High School,83.71,84.29,84.01,83.37


In [25]:
# Looking at both scores
both_scores = grade_scores.pivot(index='School',columns='Grade Level',values=['Avg Math Score','Avg Reading Score'])
both_scores.head()


Avg Math Score                      Avg Reading Score  \
Grade Level                    10th   11th   12th    9th              10th   
School                                                                       
Bailey High School            77.00  77.52  76.49  77.08             80.91   
Cabrera High School           83.15  82.77  83.28  83.09             84.25   
Figueroa High School          76.54  76.88  77.15  76.40             81.41   
Ford High School              77.67  76.92  76.18  77.36             81.26   
Griffin High School           84.23  83.84  83.36  82.04             83.71   

                                           
Grade Level            11th   12th    9th  
School                                     
Bailey High School    80.95  80.91  81.30  
Cabrera High School   83.79  84.29  83.68  
Figueroa High School  80.64  81.38  81.20  
Ford High School      80.40  80.66  80.63  
Griffin High School   84.29  84.01  83.37

### Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)


In [26]:
school_summary6['Budget per Student'].describe()

count     15.000000
mean     620.066667
std       28.544368
min      578.000000
25%      591.500000
50%      628.000000
75%      641.500000
max      655.000000
Name: Budget per Student, dtype: float64

In [27]:
# Creating Bins for Budget and Size Groups
budget_bins = [0, 580,620,640,700]
budget_labels = ['< 580','< 620',' < 640','640+']

size_bins = [0,1500,3000,5000]
size_labels = ['Small','Medium','Large']



In [28]:
# Cutting in the Bins into a new Table
school_summary7 = school_summary6

school_summary7['Budget Bin'] = pd.cut(school_summary7['Budget per Student'], budget_bins, labels=budget_labels)
school_summary7['Size Bin'] = pd.cut(school_summary7['Total Students'], size_bins, labels=size_labels)
school_summary7.sort_values(by=['Total Students'],ascending=False).reset_index(drop=True)

school_summary7.head(2)

,School Type,School,Total Students,School Budget,Average Math Score,Average Reading Score,Budget per Student,Passed Math,Passed Math Rate,Passed Reading,Passed Reading Rate,Passed Both,Passing Rate,Averge Passing Rate,Budget Bin,Size Bin
0,Charter,Cabrera High School,1858,1081356.0,83.06,83.98,582,1858,100.0,1858,100.0,1858,100.0,100.0,< 620,Medium
1,Charter,Griffin High School,1468,917500.0,83.35,83.82,625,1468,100.0,1468,100.0,1468,100.0,100.0,< 640,Small


In [29]:
school_summary7.columns

Index(['School Type', 'School', 'Total Students', 'School Budget',
       'Average Math Score', 'Average Reading Score', 'Budget per Student',
       'Passed Math', 'Passed Math Rate', 'Passed Reading',
       'Passed Reading Rate', 'Passed Both', 'Passing Rate',
       'Averge Passing Rate', 'Budget Bin', 'Size Bin'],
      dtype='object')

In [30]:
spending_cols = ['School', 
                 'Average Math Score', 'Passed Math Rate', 
                 'Average Reading Score', 'Passed Reading Rate', 
                 'Passing Rate','Budget per Student', 'Budget Bin']

spending_summary = school_summary7.groupby('Budget Bin')[spending_cols].mean()
spending_summary



,Average Math Score,Passed Math Rate,Average Reading Score,Passed Reading Rate,Passing Rate,Budget per Student
Budget Bin,,,,,,
< 580,83.2700,100.000,83.9900,100.0,100.000,578.00
< 620,83.5480,100.000,83.9020,100.0,100.000,591.00
< 640,79.4740,93.304,82.1200,100.0,93.304,633.40
640+,77.0225,89.105,80.9575,100.0,89.105,650.25


### Scores by School Size

* Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).


In [31]:
size_cols = ['School', 
                 'Average Math Score', 'Passed Math Rate', 
                 'Average Reading Score', 'Passed Reading Rate', 
                 'Passing Rate','Total Students', 'Size Bin']

size_summary = school_summary7.groupby('Size Bin')[size_cols].mean()
size_summary

,Average Math Score,Passed Math Rate,Average Reading Score,Passed Reading Rate,Passing Rate,Total Students
Size Bin,,,,,,
Small,83.663333,100.000,83.89000,100.0,100.000,952.333333
Medium,80.903750,95.825,82.82375,100.0,95.825,2242.750000
Large,77.062500,89.085,80.91750,100.0,89.085,4592.750000


### Scores by School Type

* Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).

As final considerations:

* Use the pandas library and Jupyter Notebook.
* You must submit a link to your Jupyter Notebook with the viewable Data Frames.
* You must include a written description of at least two observable trends based on the data.
* See [Example Solution](PyCitySchools/PyCitySchools_starter.ipynb) for a reference on the expected format.


In [32]:
size_cols_type = ['School', 'School Type',
                 'Average Math Score', 'Passed Math Rate', 
                 'Average Reading Score', 'Passed Reading Rate', 
                 'Passing Rate','Total Students', 'Size Bin']

size_summary_type = school_summary7.groupby(['School Type','Size Bin'])[size_cols_type].mean().dropna()
size_summary_type

Average Math Score  Passed Math Rate  \
School Type Size Bin                                         
Charter     Small              83.663333        100.000000   
            Medium             83.358000        100.000000   
District    Medium             76.813333         88.866667   
            Large              77.062500         89.085000   

                      Average Reading Score  Passed Reading Rate  \
School Type Size Bin                                               
Charter     Small                   83.8900                100.0   
            Medium                  83.9000                100.0   
District    Medium                  81.0300                100.0   
            Large                   80.9175                100.0   

                      Passing Rate  Total Students  
School Type Size Bin                                
Charter     Small       100.000000      952.333333  
            Medium      100.000000     1867.400000  
District    Medium       88.866667     2868.333333  
            Large        89.085000     4592.750000

## Hints and Considerations

* These are challenging activities for a number of reasons. For one, these activities will require you to analyze thousands of records. Hacking through the data to look for obvious trends in Excel is just not a feasible option. The size of the data may seem daunting, but pandas will allow you to efficiently parse through it.

* Second, these activities will also challenge you by requiring you to learn on your feet. Don't fool yourself into thinking: "I need to study pandas more closely before diving in." Get the basic gist of the library and then _immediately_ get to work. When facing a daunting task, it's easy to think: "I'm just not ready to tackle it yet." But that's the surest way to never succeed. Learning to program requires one to constantly tinker, experiment, and learn on the fly. You are doing exactly the _right_ thing, if you find yourself constantly practicing Google-Fu and diving into documentation. There is just no way (or reason) to try and memorize it all. Online references are available for you to use when you need them. So use them!

* Take each of these tasks one at a time. Begin your work, answering the basic questions: "How do I import the data?" "How do I convert the data into a DataFrame?" "How do I build the first table?" Don't get intimidated by the number of asks. Many of them are repetitive in nature with just a few tweaks. Be persistent and creative!

* Expect these exercises to take time! Don't get discouraged if you find yourself spending  hours initially with little progress. Force yourself to deal with the discomfort of not knowing and forge ahead. Consider these hours an investment in your future!

* As always, feel encouraged to work in groups and get help from your TAs and Instructor. Just remember, true success comes from mastery and _not_ a completed homework assignment. So challenge yourself to truly succeed!

In [71]:
grade_grid = students_schools[['Student','School Type','Math Grade','Reading Grade']]\
            .groupby(['School Type','Math Grade','Reading Grade'], as_index=False)\
            .count()\
            .dropna()\
            .sort_values('Math Grade')\
            .reset_index()\
            .drop('index',axis=1)


In [72]:
grade_grid.groupby(['School Type','Math Grade','Reading Grade']).mean().dropna().unstack(level=-1)

Student                        
Reading Grade                D       C       B       A
School Type Math Grade                                
Charter     D             26.0   253.0   237.0   252.0
            C            134.0  1253.0  1249.0  1204.0
            B            130.0  1228.0  1221.0  1220.0
            A            119.0  1213.0  1261.0  1194.0
District    F            585.0   789.0   780.0   805.0
            D           1107.0  1640.0  1677.0  1649.0
            C           1145.0  1605.0  1714.0  1544.0
            B           1107.0  1552.0  1662.0  1611.0
            A           1207.0  1596.0  1596.0  1605.0

In [ ]:
# A short program to search the grade records.
math_grade = input("What Math Grade: ")
read_grade = input('What Reading Grade: ')
school_type_search = input('What Type of School: ')

search_results = students_schools.loc[\
                     (students_schools['Math Grade']==math_grade) & \
                     (students_schools['Reading Grade']==read_grade) &\
                     (students_schools['School Type']==school_type_search)
                    ]

print(f"There are {len(search_results)} students that match.  Here is a preview:")

search_results.reset_index().head(10).drop('index', axis=1)

What Math Grade: B
What Reading Grade: B


## Woodall Observable Trends

1) From the above 'Grade Grid' Table, we can see Charter schools do not have any students failing math.  This could suggest that Charter schools have a great focus in teaching Math to ensure each student passes.

2) Big Budgets are not necessary better.  Those schools with the highest budget per student did not have the highest passing rates for their students.  In fact, what this data reveals, is that smaller Charter schools have the highest sucess rates in both Math & Reading often at the lowest budget per student.  